# Installing Whisper

The commands below will install the Python packages needed to use Whisper models and evaluate the transcription results.

In [2]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

Defaulting to user installation because normal site-packages is not writeable

  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\ckumarsingh\AppData\Local\Temp\pip-req-build-mtu2wmpp'



  Cloning https://github.com/openai/whisper.git to c:\users\ckumarsingh\appdata\local\temp\pip-req-build-mtu2wmpp
  Resolved https://github.com/openai/whisper.git to commit e8622f9afc4eba139bf796c210f5c01081000472
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
! pip3 install pandas --yes

In [ ]:
! pip install torchaudio --yes

In [ ]:
! pip install tensorflow --yes

# Loading the LibriSpeech dataset

The following will load the test-clean split of the LibriSpeech corpus using torchaudio.

In [ ]:
! pip uninstall torchaudio --yes --no-cache

In [ ]:
! pip install torchaudio

In [ ]:
! pip uninstall nvidia_cublas_cu11 --yes

In [ ]:
! pip3 install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html


In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio
from tqdm.notebook import tqdm


#DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

DEVICE = "cpu"

In [4]:
class LibriSpeech(torch.utils.data.Dataset):
    """
    A simple class to wrap LibriSpeech and trim/pad the audio to 30 seconds.
    It will drop the last few seconds of a very small portion of the utterances.
    """
    def __init__(self, split="test-clean", device=DEVICE):
        self.dataset = torchaudio.datasets.LIBRISPEECH(
            root=os.path.expanduser("~/.cache"),
            url=split,
            download=True,
        )
        self.device = device

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, item):
        audio, sample_rate, text, _, _, _ = self.dataset[item]
        assert sample_rate == 16000
        audio = whisper.pad_or_trim(audio.flatten()).to(self.device)
        mel = whisper.log_mel_spectrogram(audio)
        
        return (mel, text)

In [5]:
dataset = LibriSpeech("test-clean")
loader = torch.utils.data.DataLoader(dataset, batch_size=16)

# Running inference on the dataset using a base Whisper model

The following will take a few minutes to transcribe all utterances in the dataset.

In [6]:
model = whisper.load_model("base.en")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is English-only and has 71,825,408 parameters.


In [7]:
# predict without timestamps for short-form transcription
options = whisper.DecodingOptions(language="en", without_timestamps=True,fp16=False)

In [ ]:
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

In [ ]:
#conda install -c conda-forge pysoundfile

In [ ]:
import torchaudio

In [ ]:
str(torchaudio.get_audio_backend())

In [8]:
hypotheses = []
references = []

for mels, texts in tqdm(loader):
    results = model.decode(mels, options)
    hypotheses.extend([result.text for result in results])
    references.extend(texts)

  0%|          | 0/164 [00:00<?, ?it/s]

In [ ]:
data = pd.DataFrame(dict(hypothesis=hypotheses, reference=references))
data

# Calculating the word error rate

Now, we use our English normalizer implementation to standardize the transcription and calculate the WER.

In [ ]:
import jiwer
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

In [ ]:
data["hypothesis_clean"] = [normalizer(text) for text in data["hypothesis"]]
data["reference_clean"] = [normalizer(text) for text in data["reference"]]
data

In [ ]:
wer = jiwer.wer(list(data["reference_clean"]), list(data["hypothesis_clean"]))

print(f"WER: {wer * 100:.2f} %")